In [1]:
import whisper as w
import os

In [10]:
root_dir = os.path.split(os.getcwd())[0]
audio_path = ""
for path in os.listdir(root_dir):
    if path=="downloads":
        audio = os.listdir(os.path.join(root_dir , path))
        audio_path  = os.path.join(root_dir , path , audio[0])
audio_path

'c:\\Users\\hp\\Desktop\\python practice\\Explore\\downloads\\How to implement Decision Trees from scratch with Python.mp3'

In [11]:
print(os.path.exists(audio_path))

True


In [12]:
audio_model  = w.load_model("base")

In [13]:
transcription = audio_model.transcribe(audio_path , verbose=True)

Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:01.720]  Hey, and welcome in this lesson,
[00:01.720 --> 00:03.280]  we're going to learn how to implement
[00:03.280 --> 00:05.040]  decision trees from scratch.
[00:05.040 --> 00:07.960]  So let's first take a look at how decision trees work.
[00:07.960 --> 00:09.720]  Let's say this is a data set that we have,
[00:09.720 --> 00:13.440]  it's a data set of whether we can afford a house or not,
[00:13.440 --> 00:16.680]  and it has information of which neighborhood it is in,
[00:16.680 --> 00:17.800]  and how many rooms it has,
[00:17.800 --> 00:19.960]  and also the affordability.
[00:20.960 --> 00:24.480]  So when you want to build a decision tree, what you do
[00:24.480 --> 00:26.840]  is let's say this is all of our data sets,
[00:26.840 --> 00:29.440]  and this is basically our root note.
[00:29.440 --> 00:33.280]  So we have one, two, three, 

In [21]:
transcription["text"]

" Hey, and welcome in this lesson, we're going to learn how to implement decision trees from scratch. So let's first take a look at how decision trees work. Let's say this is a data set that we have, it's a data set of whether we can afford a house or not, and it has information of which neighborhood it is in, and how many rooms it has, and also the affordability. So when you want to build a decision tree, what you do is let's say this is all of our data sets, and this is basically our root note. So we have one, two, three, four, five, six, seven, eight, data points here representing all these data points here. So what we say is at first, okay, we want to divide which neighborhood this house is in. So we can say if it's in the East neighborhood, here's where all the data points are going, if it's in the West neighborhood, here's where all the data points are going. And then you can do it again for the other column, you can say, okay, how many number of rooms, but you don't do it for ea

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000 , chunk_overlap = 200)
text  = text_splitter.split_text(transcription["text"])

In [29]:
embeddings_model = OllamaEmbeddings(model="nomic-embed-text:latest")
db = Chroma.from_texts(text , embeddings_model)
retriever = db.as_retriever()

In [31]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import PromptTemplate
from langchain_google_genai.llms import GoogleGenerativeAI

In [36]:
prompt = PromptTemplate(
    template="""
You are an expert who knows about everything , 
Answer the user queries based on the given context which is based on a youtube video and
also enhance the answers before giving it.

<context>
{context}
</context>

<question>
{input}
</questions>

""", input_variables=["context" , "input"]
)

In [37]:
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"]  = os.getenv("GEMINI_API_KEY")

llm = GoogleGenerativeAI(model="gemini-2.0-flash")

In [38]:
combine_chain = create_stuff_documents_chain(llm = llm , prompt=prompt)
retrieval_chain  = create_retrieval_chain(retriever=retriever , combine_docs_chain=combine_chain)


In [39]:
res = retrieval_chain.invoke({"input":"Summarize the video's content in 100 words"})

In [40]:
res["answer"]

"This video explains the process of building a decision tree. It uses NumPy's `bincount` function to create a histogram of value occurrences, which helps calculate the probability of each value. The video details how the tree is built during training by calculating the information gain from each potential split (feature and value) and dividing the dataset accordingly. This process repeats for newly created nodes until a stopping criterion is met. The video also mentions that the code used is available in the description.\n"

"This video explains the process of building a decision tree. It uses NumPy's `bincount` function to create a histogram of value occurrences, which helps calculate the probability of each value. The video details how the tree is built during training by calculating the information gain from each potential split (feature and value) and dividing the dataset accordingly. This process repeats for newly created nodes until a stopping criterion is met. The video also mentions that the code used is available in the description.\n"